In [1]:
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
import numpy as np
from sklearn.neighbors import LocalOutlierFactor
pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.max_columns', None)

df = pd.read_csv('cancer_classification.csv', sep = ',')

In [2]:
df.describe()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,radius error,texture error,perimeter error,area error,smoothness error,compactness error,concavity error,concave points error,symmetry error,fractal dimension error,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,benign_0__mal_1
count,569.00,569.00,569.00,569.00,569.00,569.00,569.00,569.00,569.00,569.00,569.00,569.00,569.00,569.00,569.00,569.00,569.00,569.00,569.00,569.00,569.00,569.00,569.00,569.00,569.00,569.00,569.00,569.00,569.00,569.00,569.00
mean,14.13,19.29,91.97,654.89,0.10,0.10,0.09,0.05,0.18,0.06,0.41,1.22,2.87,40.34,0.01,0.03,0.03,0.01,0.02,0.00,16.27,25.68,107.26,880.58,0.13,0.25,0.27,0.11,0.29,0.08,0.63
std,3.52,4.30,24.30,351.91,0.01,0.05,0.08,0.04,0.03,0.01,0.28,0.55,2.02,45.49,0.00,0.02,0.03,0.01,0.01,0.00,4.83,6.15,33.60,569.36,0.02,0.16,0.21,0.07,0.06,0.02,0.48
min,6.98,9.71,43.79,143.50,0.05,0.02,0.00,0.00,0.11,0.05,0.11,0.36,0.76,6.80,0.00,0.00,0.00,0.00,0.01,0.00,7.93,12.02,50.41,185.20,0.07,0.03,0.00,0.00,0.16,0.06,0.00
25%,11.70,16.17,75.17,420.30,0.09,0.06,0.03,0.02,0.16,0.06,0.23,0.83,1.61,17.85,0.01,0.01,0.02,0.01,0.02,0.00,13.01,21.08,84.11,515.30,0.12,0.15,0.11,0.06,0.25,0.07,0.00
50%,13.37,18.84,86.24,551.10,0.10,0.09,0.06,0.03,0.18,0.06,0.32,1.11,2.29,24.53,0.01,0.02,0.03,0.01,0.02,0.00,14.97,25.41,97.66,686.50,0.13,0.21,0.23,0.10,0.28,0.08,1.00
75%,15.78,21.80,104.10,782.70,0.11,0.13,0.13,0.07,0.20,0.07,0.48,1.47,3.36,45.19,0.01,0.03,0.04,0.01,0.02,0.00,18.79,29.72,125.40,1084.00,0.15,0.34,0.38,0.16,0.32,0.09,1.00
max,28.11,39.28,188.50,2501.00,0.16,0.35,0.43,0.20,0.30,0.10,2.87,4.88,21.98,542.20,0.03,0.14,0.40,0.05,0.08,0.03,36.04,49.54,251.20,4254.00,0.22,1.06,1.25,0.29,0.66,0.21,1.00


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   mean radius              569 non-null    float64
 1   mean texture             569 non-null    float64
 2   mean perimeter           569 non-null    float64
 3   mean area                569 non-null    float64
 4   mean smoothness          569 non-null    float64
 5   mean compactness         569 non-null    float64
 6   mean concavity           569 non-null    float64
 7   mean concave points      569 non-null    float64
 8   mean symmetry            569 non-null    float64
 9   mean fractal dimension   569 non-null    float64
 10  radius error             569 non-null    float64
 11  texture error            569 non-null    float64
 12  perimeter error          569 non-null    float64
 13  area error               569 non-null    float64
 14  smoothness error         5

In [4]:
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
import numpy as np
from sklearn.neighbors import LocalOutlierFactor
pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.max_columns', None)

df = pd.read_csv('cancer_classification.csv', sep = ',')


df_correlacao = df.corr().iloc[:,-1:]
features_com_corr = df_correlacao.loc[(df_correlacao['benign_0__mal_1'] >= 0.5) | (df_correlacao['benign_0__mal_1'] <= -0.5)].iloc[:-1,:]
feature_list = list(features_com_corr.index)

x = df[feature_list]
y = df['benign_0__mal_1']

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures


X = df[feature_list]
y = df['benign_0__mal_1']


x_train, x_, y_train, y_ = train_test_split(x, y, test_size=0.40, random_state=1)

x_cv, x_test, y_cv, y_test = train_test_split(x_, y_, test_size=0.50, random_state=1)


poly = PolynomialFeatures(degree=2, include_bias=False)
scaler_poly = StandardScaler()



# Add polynomial features and scale the train set
X_train_mapped = poly.fit_transform(x_train)
X_train_mapped_scaled = scaler_poly.fit_transform(X_train_mapped)

# Add polynomial features and scale the cross validation set
X_cv_mapped = poly.transform(x_cv)
X_cv_mapped_scaled = scaler_poly.transform(X_cv_mapped)

# Add polynomial features and scale the test set
X_test_mapped = poly.transform(x_test)
X_test_mapped_scaled = scaler_poly.transform(X_test_mapped)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf

model = Sequential(
    [ 
        Dense(15, activation = 'relu'),
        Dense(15, activation = 'relu'),
        Dense(1, activation = 'linear')  
    ]
)
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy', 'mse']
)

model.fit(
    X_train_mapped_scaled,y_train,
    epochs=100
)

from sklearn.metrics import accuracy_score, classification_report, mean_squared_error

y_pred_train = model.predict(X_test_mapped_scaled)
prob = tf.nn.sigmoid(y_pred_train)

mse = mean_squared_error(y_test,prob)

Epoch 1/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4940 - loss: 0.5375 - mse: 0.7797
Epoch 2/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 801us/step - accuracy: 0.9484 - loss: 0.2938 - mse: 2.6474
Epoch 3/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 801us/step - accuracy: 0.9211 - loss: 0.2049 - mse: 7.3071
Epoch 4/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 701us/step - accuracy: 0.9391 - loss: 0.1439 - mse: 12.3406
Epoch 5/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 801us/step - accuracy: 0.9214 - loss: 0.1568 - mse: 19.1940
Epoch 6/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 701us/step - accuracy: 0.9342 - loss: 0.1390 - mse: 22.7419
Epoch 7/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 701us/step - accuracy: 0.9306 - loss: 0.1322 - mse: 23.5481
Epoch 8/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 701us/step - accuracy: 0.9305 - loss: 0.1380 - mse: 24.5621
Epoch 9/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 701us/step - accuracy: 0.9162 - loss: 0.1539 - mse: 29.4776
Epoch 10/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 701us/step - accuracy: 0.9252 - loss: 0.1309

In [5]:
x = df[feature_list]
y = df['benign_0__mal_1']

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures


X = df[feature_list]
y = df['benign_0__mal_1']


x_train, x_, y_train, y_ = train_test_split(x, y, test_size=0.40, random_state=1)

x_cv, x_test, y_cv, y_test = train_test_split(x_, y_, test_size=0.50, random_state=1)


poly = PolynomialFeatures(degree=2, include_bias=False)
scaler_poly = StandardScaler()



# Add polynomial features and scale the train set
X_train_mapped = poly.fit_transform(x_train)
X_train_mapped_scaled = scaler_poly.fit_transform(X_train_mapped)

# Add polynomial features and scale the cross validation set
X_cv_mapped = poly.transform(x_cv)
X_cv_mapped_scaled = scaler_poly.transform(X_cv_mapped)

# Add polynomial features and scale the test set
X_test_mapped = poly.transform(x_test)
X_test_mapped_scaled = scaler_poly.transform(X_test_mapped)

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf

model = Sequential(
    [ 
        Dense(15, activation = 'relu'),
        Dense(15, activation = 'relu'),
        Dense(1, activation = 'linear')  
    ]
)
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy', 'mse']
)

model.fit(
    X_train_mapped_scaled,y_train,
    epochs=100
)

from sklearn.metrics import accuracy_score, classification_report, mean_squared_error

y_pred_train = model.predict(X_test_mapped_scaled)
prob = tf.nn.sigmoid(y_pred_train)

Epoch 1/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 801us/step - accuracy: 0.3933 - loss: 0.7403 - mse: 3.4310
Epoch 2/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 701us/step - accuracy: 0.3618 - loss: 0.4802 - mse: 11.4752
Epoch 3/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 701us/step - accuracy: 0.6678 - loss: 0.3761 - mse: 24.5534
Epoch 4/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 701us/step - accuracy: 0.9130 - loss: 0.2956 - mse: 24.5658
Epoch 5/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 701us/step - accuracy: 0.9354 - loss: 0.2162 - mse: 36.9983
Epoch 6/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 801us/step - accuracy: 0.9293 - loss: 0.1826 - mse: 36.5401
Epoch 7/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 701us/step - accuracy: 0.9331 - loss: 0.1779 - mse: 32.7535
Epoch 8/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 701us/step - accuracy: 0.9496 - loss: 0.1344 - mse: 29.0169
Epoch 9/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 701us/step - accuracy: 0.9209 - loss: 0.1756 - mse: 32.9497
Epoch 10/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 801us/step - accuracy: 0.9498 - loss: 0.

In [8]:
mse = mean_squared_error(y_test,prob)
mse

0.022650244130615834

In [10]:
import joblib

# Supondo que 'model' seja o seu modelo treinado
# model.fit(X_train, y_train)

# Salvar o modelo como um arquivo .pkl
joblib.dump(model, 'model_neural_class.pkl')

['model_neural_class.pkl']